<a href="https://colab.research.google.com/github/CorentinGaillard/GTNH_craft_calc/blob/main/GTNH_calc_on_gsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTNH Calculator

It takes in a csv with all the recipe you are interested in, and outputs a list of all the ressources you will need

In [1]:
import pandas as pd, numpy as np, requests
from collections import defaultdict
from json import dumps

In [ ]:
# reading the csv file with the recipes from github:
file = "https://raw.githubusercontent.com/CorentinGaillard/GTNH_craft_calc/main/GTNH%20circuits%20-%20recipe.csv"
df = pd.read_csv(file)

In [3]:
# Other alternative : read from google sheet
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [5]:
worksheet = gc.open_by_key('1xY8lWg1q8Nwm145chN9CTzGZMhrCRxYMlNVZgBhbaYM').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
pd.DataFrame.from_records(rows)
# link = "https://docs.google.com/spreadsheets/d/1xY8lWg1q8Nwm145chN9CTzGZMhrCRxYMlNVZgBhbaYM/edit?usp=sharing"


,0,1,2,3,4,5,6,7,8,9,10
0,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity,Byproduct,quantity
1,Advanced Circuit,1,30,24000,800,40,Circuit assembler,Good Intergrated Circuit,1,,
2,,,,,,,,Integrated Logic Circuit,2,,
3,,,,,,,,Random Access Memory Chip,2,,
4,,,,,,,,SMD Transistor,4,,
...,...,...,...,...,...,...,...,...,...,...,...
89,,,,,,,,Programmed Cuircuit (2) (Not consumed),1,,
90,Refined Glue,100,5,1500,300,15,centrifuge,Sticky resin,1,Raw rubber dust,3
91,Annealed Copper Ingot,1,30,5670,189,9.45,arc furnace,copper ingot,1,,
92,,,,,,,,oxygen gas,63,,


In [ ]:
# Filling in NaNs
tmp = df.drop(columns=['Byproduct', 'quantity.2']).fillna(method='ffill')
tmp['Byproduct'] = df.Byproduct
tmp['quantity.2'] = df['quantity.2']
df = tmp
del tmp
df

,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2
0,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Good Intergrated Circuit,1,NaN,NaN
1,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Integrated Logic Circuit,2,NaN,NaN
2,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Random Access Memory Chip,2,NaN,NaN
3,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,SMD Transistor,4,NaN,NaN
4,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Fine electrum Wire,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
88,Gallium arsenide dust,2.0,30.0,9000.0,300.0,15.00,Mixer,Programmed Cuircuit (2) (Not consumed),1,NaN,NaN
89,Refined Glue,100.0,5.0,1500.0,300.0,15.00,centrifuge,Sticky resin,1,Raw rubber dust,3.0
90,Annealed Copper Ingot,1.0,30.0,5670.0,189.0,9.45,arc furnace,copper ingot,1,NaN,NaN
91,Annealed Copper Ingot,1.0,30.0,5670.0,189.0,9.45,arc furnace,oxygen gas,63,NaN,NaN


In [ ]:
def to_lower(serie):
  try:
     return serie.str.lower()
  except: ...
  return serie
df = df.apply(to_lower)

In [ ]:
def get(nb_item, name_item, df, mult=1):
  global ingredients, intermediates
  # If no recipe, return the name and the quantity
  if not name_item in df.obj.values:
    return  (nb_item*mult, name_item)
  # else, get the ingredients for the subcraft, and call the function on each ingredient
  componants = df.loc[df.obj==name_item, ["ressources in", "quantity.1"]]
  for name, nb in componants.values:
    qn = get(nb, name, df, mult=nb_item*mult/df.loc[df.obj==name_item, "quantity"].values[0])
    if qn:
    # if (qn := get(nb, name, df, mult=nb_item*mult/df.loc[df.obj==name_item, "quantity"].values[0])): # Python 3.9 is not here yet TT
      # if the function return, it means that there is no recipe for this craft, thus add it to the ingredients dict
      ingredients[qn[1]] += qn[0] #* mult
  intermediates[name_item] += nb_item*mult

In [ ]:
def get_quantity_and_recipe(name, nb_item):
  global ingredients, intermediates
  ingredients = defaultdict(int)
  intermediates = defaultdict(int)
  get(nb_item, name, df)
  def boolify(d):
    for name in d.keys():
      if '(not consumed)' in name:
        d[name] = int(bool(d[name]))
  boolify(ingredients)
  boolify(intermediates)
  idx = df.loc[df.obj==intermediates.keys()].index
  for name, nb in intermediates.items():
    idx = np.r_[idx, df.loc[df.obj==name].index]
    mask = df.obj==name
    df.loc[mask, "quantity to craft"] = nb
    df.loc[mask, "total eu craft"] = df.loc[mask, "total eu"] * nb
    df.loc[mask, "total seconds craft"] = df.loc[mask, "total seconds"] * nb
  return df.loc[idx]

In [ ]:
# The available recipe are:
df.obj.unique()

array(['advanced circuit', 'good intergrated circuit',
       'integrated logic circuit', 'smd transistor', 'smd resistor',
       'diode', 'random access memory chip',
       'integrated logic circuit chip', 'good circuit board',
       'circuit board', 'random access memory chip (wafer)',
       'integrated logic circuit (wafer)', 'phenolic circuit board',
       'fine electrum wire', 'electrum wire', 'annealed copper bolt',
       'silver bolt', 'fine copper wire', 'copper wire', 'tin bolt',
       'gallium foil', 'gallium plate', 'fine annealed copper wire',
       'annealed copper wire', 'gold foil', 'gold plate',
       'iron iii chloride', 'hydrocholric acid', 'chlorine', 'wood plank',
       'copper foil', 'copper plate', 'wood pulp', 'molten tin', 'wafer',
       'monocrystalline silicon boule',
       'small pile of gallium arsenide dust', 'gallium arsenide dust',
       'refined glue', 'annealed copper ingot', 'gallium ingot'],
      dtype=object)

In [ ]:
# to get the componant for a recipe, you call the function, 
# then you look into df and ingredients
out = get_quantity_and_recipe("advanced circuit", 64)
print(dumps(ingredients, indent=2))
out

{
  "log": 181.33333333333331,
  "sticky resin": 23.04,
  "programmed cuircuit (1) (not consumed)": 1,
  "gold ingot": 64.0,
  "iron dust": 6.4,
  "empty cell ": 19.200000000000003,
  "hydrogen cell": 19.2,
  "rock salt": 38.4,
  "copper ingot": 368.0,
  "molten polyethylene": 50688.0,
  "silicon dust": 456.0,
  "gallium dust": 5.78125,
  "arsenic dust": 1.78125,
  "programmed cuircuit (2) (not consumed)": 1,
  "water": 7367.25,
  "ruby lens (not consumed)": 1,
  "carbon dust": 48.0,
  "electrum ingot": 120.0,
  "oxygen gas": 11088.0,
  "tin ingot": 448.0,
  "extruder shape (bolt) (not consumed)": 1,
  "silver ingot": 32.0,
  "green sapphire lens (not consumed)": 1
}


,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2,quantity to craft,total eu craft,total seconds craft
79,wood pulp,6.0,2.0,800.0,400.0,20.0,pulverization,log,1,NaN,NaN,1088.0,870400.0,21760.0
89,refined glue,100.0,5.0,1500.0,300.0,15.0,centrifuge,sticky resin,1,raw rubber dust,3.0,2304.0,3456000.0,34560.0
44,phenolic circuit board,1.0,16.0,4800.0,300.0,15.0,assembler,wood pulp,1,NaN,NaN,64.0,307200.0,960.0
45,phenolic circuit board,1.0,16.0,4800.0,300.0,15.0,assembler,refined glue,36,NaN,NaN,64.0,307200.0,960.0
46,phenolic circuit board,1.0,16.0,4800.0,300.0,15.0,assembler,programmed cuircuit (1) (not consumed),1,NaN,NaN,64.0,307200.0,960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,advanced circuit,1.0,30.0,24000.0,800.0,40.0,circuit assembler,random access memory chip,2,NaN,NaN,64.0,1536000.0,2560.0
3,advanced circuit,1.0,30.0,24000.0,800.0,40.0,circuit assembler,smd transistor,4,NaN,NaN,64.0,1536000.0,2560.0
4,advanced circuit,1.0,30.0,24000.0,800.0,40.0,circuit assembler,fine electrum wire,8,NaN,NaN,64.0,1536000.0,2560.0
5,advanced circuit,1.0,30.0,24000.0,800.0,40.0,circuit assembler,annealed copper bolt,8,NaN,NaN,64.0,1536000.0,2560.0


In [ ]:
# tests
out = get_quantity_and_recipe("wood plank", 64)
print(dumps(ingredients, indent=2))
out
# 1 log = 6 pulp, 8 pulp = 1 plank -> 1 log = 6/8 = 0.75 plank
# 1 plank = 8/6 = 1.333... log
# 64 plank = 64* 8/6 = 85.3333 log OK

{
  "log": 85.33333333333333
}


,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2,quantity to craft,total eu craft,total seconds craft
79,wood pulp,6.0,2.0,800.0,400.0,20.0,pulverization,log,1,NaN,NaN,512.0,409600.0,10240.0
74,wood plank,1.0,2.0,600.0,300.0,15.0,compressor,wood pulp,8,NaN,NaN,64.0,38400.0,960.0


In [ ]:
# tests
out = get_quantity_and_recipe("phenolic circuit board", 64)
print(dumps(ingredients, indent=2))
out
# 1 board = 36 glue
# 64 board = 36*64 = 2304 glue OK

{
  "log": 10.666666666666666,
  "sticky resin": 23.04,
  "programmed cuircuit (1) (not consumed)": 1
}


,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2,quantity to craft,total eu craft,total seconds craft
79,wood pulp,6.0,2.0,800.0,400.0,20.0,pulverization,log,1,NaN,NaN,64.0,51200.0,1280.0
89,refined glue,100.0,5.0,1500.0,300.0,15.0,centrifuge,sticky resin,1,raw rubber dust,3.0,2304.0,3456000.0,34560.0
44,phenolic circuit board,1.0,16.0,4800.0,300.0,15.0,assembler,wood pulp,1,NaN,NaN,64.0,307200.0,960.0
45,phenolic circuit board,1.0,16.0,4800.0,300.0,15.0,assembler,refined glue,36,NaN,NaN,64.0,307200.0,960.0
46,phenolic circuit board,1.0,16.0,4800.0,300.0,15.0,assembler,programmed cuircuit (1) (not consumed),1,NaN,NaN,64.0,307200.0,960.0
